In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import importlib
import torch
import torchvision.transforms as transforms
from mmcv import Config
from mmcls.datasets.pipelines import Compose
from mmcls.models.builder import build_classifier
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from matplotlib import cm
from PIL import Image
import heapq
from mmseg.apis import inference_segmentor, init_segmentor
from mmseg.core.evaluation import get_palette
from mmseg.datasets.builder import build_dataset, build_dataloader, DATASETS
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2
import pandas
import json
import seaborn as sns

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

from scripts import generate_statistics_single
from scripts import visualization_seg_masks
from scripts import generate_statistics
from scripts import generate_cams
from scripts import utils
from scripts.utils.pipeline import get_pipeline_torchvision
from scripts import generate_segs
from scripts import visualize_cam_results
from scripts.utils.BlurDataset import get_blurred_dataset
from scripts.utils import preprocessing, io
from scripts.utils.model import get_wrongly_classified
from scripts.utils.evaluation import compare_original_blurred, get_eval_metrics
from scripts import evaluate_effectiveness

Importing BlurSegments step into Pipeline


In [3]:
configSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/ocrnet_hr48_512x512_carparts_noflip.py'
checkpointSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/latest.pth'

segDataPathOriginal = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegs.npz'
segDataPathColor = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color/valAllSegs.npz'
segDataPathWeb = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web/valAllSegs.npz'
segDataPathStanford = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegs.npz'

paletteName='Comp_Original_Ocrnet_Carparts_Noflip'

segDataPathOG = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegs.npz'
segDataPathNoScaleOG = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegsNoScale.npz'
segDataPathNoScaleCol = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color/valAllSegsNoScale.npz'
segDataPathNoScaleWeb = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web/valAllSegsNoScale.npz'
segDataPathNoScaleStanford = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegsNoScale.npz'

camDataGradCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMs.npz'
camDataEigenCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsEigen.npz'
camDataEigenGrad = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsEigenGrad.npz'
camDataGradCAMGPU = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsGPU.npz'
camDataGradCAMPlusPlus = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsGrad++.npz'
camDataLayerCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsLayer.npz'
camDataXCAM = 'G:/XAI-Vehicle-Classification/preGenData/CompCars_original/ResNet/valAllCAMsXGrad.npz'

imgRootValOriginal = '../data/CompCars_sv_original_split/val'
imgRootValColor = '../data/CompCars_sv_color_split/val'
imgRootValWeb = '../data/CompCars_web_original_split/val'
imgRootValStandford = '../data/StanfordCars_original_split/val'

blurOGBackground = '../data/CompCars_sv_original_split/blurred/blurredBackground'
blurOGBackground_Hood_FBumper = '../data/CompCars_sv_original_split/blurred/blurredBackground_Hood_FrontBumper'
blurOGFGlass_Hood_FBumper = '../data/CompCars_sv_original_split/blurred/blurredFrontGlass_Hood_FrontBumper'
blurOGFLLight_Hood_FBumper = '../data/CompCars_sv_original_split/blurred/blurredFrontLeftLight_Hood_FrontBumper'

blurColBackground = '../data/CompCars_sv_color_split/blurred/blurredBackground'
blurColBackground_Hood_FBumper = '../data/CompCars_sv_color_split/blurred/blurredBackground_Hood_FrontBumper'
blurColFGlass_Hood_FBumper = '../data/CompCars_sv_color_split/blurred/blurredFrontGlass_Hood_FrontBumper'
blurColFLLight_Hood_FBumper = '../data/CompCars_sv_color_split/blurred/blurredFrontLeftLight_Hood_FrontBumper'

blurWebBackground = '../data/CompCars_web_original_split/blurred/blurredBackground'
blurWebBBumper_Background_FBumper = '../data/CompCars_web_original_split/blurred/blurredBackBumper_Background_FrontBumper'
blurWebBackground_Hood_FBumper = '../data/CompCars_web_original_split/blurred/blurredBackground_Hood_FrontBumper'

annfileValOriginal = '../data/CompCars_sv_original_split/meta/val.txt'
annfileValColor = '../data/CompCars_sv_color_split/meta/val.txt'
annfileValWeb = '../data/CompCars_web_original_split/meta/val.txt'
annfileValStanford = '../data/StanfordCars_original_split/meta/val.txt'

dirResOG = 'G:/results/CompCars_original/ResNet'
dirResCol = 'G:/results/CompCars_Color/ResNet'
dirResWeb = 'G:/results/CompCars_Web/ResNet'

dirSwinBaseOG = 'G:/results/CompCars_original/SwinBase'
dirSwinBaseCol = 'G:/results/CompCars_Color/SwinBase'
dirSwinBaseWeb = 'G:/results/CompCars_Web/SwinBase'

dirSwinSmallOG = 'G:/results/CompCars_original/SwinSmall'
dirSwinSmallCol = 'G:/results/CompCars_Color/SwinSmall'
dirSwinSmallWeb = 'G:/results/CompCars_Web/SwinSmall'

configCAMResOriginal = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
checkpointCAMResOriginal = '../CAMModels/resnet/compCars_Original/latest.pth'
configCAMResOGAdam = '../CAMModels/resnet/compCars_Original/adamw/resnet50_b128x2_compcars-original-split_adamw.py'
checkpointCAMResOGAdam = '../CAMModels/resnet/compCars_Original/adamw/latest.pth'
configCAMResColor = '../CAMModels/resnet/compCars_Color/resnet50_b128x2_compcars-color-split.py'
checkpointCAMResColor = '../CAMModels/resnet/compCars_Color/latest.pth'
configCAMResWeb = '../CAMModels/resnet/compCars_Web/resnet50_b128x2_compcars-web-original-split.py'
checkpointCAMResWeb = '../CAMModels/resnet/compCars_Web/latest.pth'
configCAMResWebAdam = '../CAMModels/resnet/compCars_Web/adamw/resnet50_b128x2_compcars-web-original-split_adamw.py'
checkpointCAMResWebAdam = '../CAMModels/resnet/compCars_Web/adamw/latest.pth'
configCAMResStanford = '../CAMModels/resnet/stanfordCars_Original/resnet50_b128x2_stanford.py'
checkpointCAMResStanford = '../CAMModels/resnet/stanfordCars_Original/latest.pth'

configCAMSwinBaseOriginal = '../CAMModels/swinBase/compCars_Original/swin-base_b128x2_compcars-original-split.py'
checkpointCAMSwinBaseOriginal = '../CAMModels/swinBase/compCars_Original/latest.pth'
configCAMSwinBaseColor = '../CAMModels/swinBase/compCars_Color/swin-base_b128x2_compcars-color-split.py'
checkpointCAMSwinBaseColor = '../CAMModels/swinBase/compCars_Color/latest.pth'
configCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_Web/swin-base_b128x2_compcars-web-original-split.py'
checkpointCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_Web/latest.pth'
configCAMSwinBaseStanford = '../CAMModels/swinBase/stanfordCars_Original/swin-base_b128x2_stanford.py'
checkpointCAMSwinBaseStanford = '../CAMModels/swinBase/stanfordCars_Original/latest.pth'

configCAMSwinSmallOriginal = '../CAMModels/swinSmall/compCars_Original/swin-small_b128x2_compcars-original-split.py'
checkpointCAMSwinSmallOriginal = '../CAMModels/swinSmall/compCars_Original/latest.pth'
configCAMSwinSmallColor = '../CAMModels/swinSmall/compCars_Color/swin-small_b128x2_compcars-color-split.py'
checkpointCAMSwinSmallColor = '../CAMModels/swinSmall/compCars_Color/latest.pth'
configCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_Web/swin-small_b128x2_compcars-web-original-split.py'
checkpointCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_Web/latest.pth'
configCAMSwinSmallStanford = '../CAMModels/swinSmall/stanfordCars_Original/swin-small_b128x2_stanford.py'
checkpointCAMSwinSmallStanford = '../CAMModels/swinSmall/stanfordCars_Original/latest.pth'

eval_data_original_res = 'G:/results/CompCars_original/eval_results_resnet.json'
eval_data_original_swinbase = 'G:/results/CompCars_original/eval_results_swinbase.json'
eval_data_original_swinsmall = 'G:/results/CompCars_original/eval_results_swinsmall.json'

eval_data_color_res = 'G:/results/CompCars_Color/eval_results_resnet.json'
eval_data_color_swinbase = 'G:/results/CompCars_Color/eval_results_swinbase.json'
eval_data_color_swinsmall = 'G:/results/CompCars_Color/eval_results_swinsmall.json'

eval_data_web_res = 'G:/results/CompCars_Web/eval_results_resnet.json'
eval_data_web_swinbase = 'G:/results/CompCars_Web/eval_results_swinbase.json'
eval_data_web_swinsmall = 'G:/results/CompCars_Web/eval_results_swinsmall.json'

eval_data_stanford_res = 'G:/results/StanfordCars_Original/eval_results_resnet.json'
eval_data_stanford_swinbase = 'G:/results/StanfordCars_Original/eval_results_swinbase.json'
eval_data_stanford_swinsmall = 'G:/results/StanfordCars_Original/eval_results_swinsmall.json'

In [ ]:
"""CompCars_Original"""
"""ResNet"""
# Eigencam
baseExcel = os.path.join(dirResOG, 'normalized')
excel = os.path.join(baseExcel,'Data_Full_EigenCAM_ResNet_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/ResNet/blurRelImpEigen', 
                                                eval_data_original=eval_data_original_res,
                                                importanceScoreFile=excel)

excel = os.path.join(baseExcel,'Data_Full_gradCAM++_ResNet_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/ResNet/blurRelImpGrad++', 
                                                eval_data_original=eval_data_original_res,
                                                importanceScoreFile=excel)

"""SwinBase"""
# Eigencam
baseExcel = os.path.join(dirSwinBaseOG, 'normalized')
excel = os.path.join(baseExcel,'Data_Full_EigenCAM_SwinBase_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinBase/blurRelImpEigen', 
                                                eval_data_original=eval_data_original_swinbase,
                                                importanceScoreFile=excel)

excel = os.path.join(baseExcel,'Data_Full_gradCAM++_SwinBase_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinBase/blurRelImpGrad++', 
                                                eval_data_original=eval_data_original_swinbase,
                                                importanceScoreFile=excel)

"""SwinSmall"""
# Eigencam
baseExcel = os.path.join(dirSwinSmallOG, 'normalized')
excel = os.path.join(baseExcel,'Data_Full_EigenCAM_SwinSmall_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinSmall/blurRelImpEigen', 
                                                eval_data_original=eval_data_original_swinsmall,
                                                importanceScoreFile=excel)

excel = os.path.join(baseExcel,'Data_Full_gradCAM++_SwinSmall_CompCars_Original_ocrnet_hr48_carparts_noflip_normalized_PropArea_23_08_2022.xlsx')
evaluate_effectiveness.evaluate_blurred_normalized_by_rel_importance(
                                                imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinSmall/blurRelImpGrad++', 
                                                eval_data_original=eval_data_original_swinsmall,
                                                importanceScoreFile=excel)

In [7]:
"""Blurred Background White"""
# CompCars_Original
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/ResNet/backgroundWhite', 
                                                eval_data_original=eval_data_original_res, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootOriginal)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinBase/backgroundWhite', 
                                                eval_data_original=eval_data_original_swinbase, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootOriginal)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Original/SwinSmall/backgroundWhite', 
                                                eval_data_original=eval_data_original_swinsmall, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootOriginal)

# CompCars_Web
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMResWeb, 
                                                classifierCheckpoint=checkpointCAMResWeb, annfile=annfileValWeb,
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Web/ResNet/backgroundWhite', 
                                                eval_data_original=eval_data_web_res, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootWeb)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMSwinBaseWeb, 
                                                classifierCheckpoint=checkpointCAMSwinBaseWeb, annfile=annfileValWeb,
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Web/SwinBase/backgroundWhite', 
                                                eval_data_original=eval_data_web_swinbase, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootWeb)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMSwinSmallWeb, 
                                                classifierCheckpoint=checkpointCAMSwinSmallWeb, annfile=annfileValWeb,
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/CompCars_Web/SwinSmall/backgroundWhite', 
                                                eval_data_original=eval_data_web_swinsmall, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootWeb)

# StanfordCars_Original
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValStandford, classifierConfig=configCAMResStanford, 
                                                classifierCheckpoint=checkpointCAMResStanford, annfile=annfileValStanford,
                                                segData=segDataPathNoScaleStanford, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/StanfordCars_Original/ResNet/backgroundWhite', 
                                                eval_data_original=eval_data_stanford_res, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootStanford)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValStandford, classifierConfig=configCAMSwinBaseStanford, 
                                                classifierCheckpoint=checkpointCAMSwinBaseStanford, annfile=annfileValStanford,
                                                segData=segDataPathNoScaleStanford, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/StanfordCars_Original/SwinBase/backgroundWhite', 
                                                eval_data_original=eval_data_stanford_swinbase, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootStanford)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValStandford, classifierConfig=configCAMSwinSmallStanford, 
                                                classifierCheckpoint=checkpointCAMSwinSmallStanford, annfile=annfileValStanford,
                                                segData=segDataPathNoScaleStanford, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='results/StanfordCars_Original/SwinSmall/backgroundWhite', 
                                                eval_data_original=eval_data_stanford_swinsmall, 
                                                imgRootBlurred=blurWhiteBackgroundImgRootStanford)

Evaluating original model vs blurred background.
Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at results/CompCars_Original/ResNet/backgroundWhite\annfile_filtered.txt
Using standard dataset on blurred data at directory ../data/CompCars_sv_original_split/blurred/blurredBackgroundWhite
Model already on GPU
Using given evaluation data of original model.
Loading data from Json G:/results/CompCars_original/eval_results_resnet.json
Computing Evaluation Metrics for Blurred Dataset
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 242.3 task/s, elapsed: 55s, ETA:     0s
Saving json data to results/CompCars_Original/ResNet/backgroundWhite\eval_results_blurred.json
Add total Change and improvement of original over blurred
Saving evaluation results to results/CompCars_Original/ResNet/backgroundWhite\evalBlurred.xlsx
Removing temporary filtered annotation file results/CompCars_Original/ResNet/backgroundWhite\annfile_filtered.txt
Evaluating orig

In [6]:
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMResOriginal, 
                                                classifierCheckpoint=checkpointCAMResOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='res', 
                                                eval_data_original=eval_data_original_res, 
                                                imgRootBlurred='blurredBackgroundN')
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinBaseOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinBaseOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='swinbase', 
                                                eval_data_original=eval_data_original_swinbase, 
                                                imgRootBlurred='blurredBackgroundN')
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValOriginal, classifierConfig=configCAMSwinSmallOriginal, 
                                                classifierCheckpoint=checkpointCAMSwinSmallOriginal, annfile=annfileValOriginal,
                                                segData=segDataPathNoScaleOG, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='swinsmall', 
                                                eval_data_original=eval_data_original_swinsmall, 
                                                imgRootBlurred='blurredBackgroundN')

Evaluating original model vs blurred background.
Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at res\annfile_filtered.txt
Using standard dataset on blurred data at directory blurredBackgroundN
Model already on GPU
Using given evaluation data of original model.
Loading data from Json G:/results/CompCars_original/eval_results_resnet.json
Computing Evaluation Metrics for Blurred Dataset
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 273.5 task/s, elapsed: 49s, ETA:     0s
Saving json data to res\eval_results_blurred.json
Add total Change and improvement of original over blurred
Saving evaluation results to res\evalBlurred.xlsx
Removing temporary filtered annotation file res\annfile_filtered.txt
Evaluating original model vs blurred background.
Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at swinbase\annfile_filtered.txt
Using standard dataset on blurred data at directory

In [3]:
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMSwinBaseWeb, 
                                                classifierCheckpoint=checkpointCAMSwinBaseWeb, 
                                                annfile='../data/CompCars_web_original_split/meta/valKopie.txt',
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='swinBase', 
                                                eval_data_original=eval_data_web_swinbase, 
                                                imgRootBlurred=blurBackgroundImgRootWeb)
evaluate_effectiveness.evaluate_blurred_background(imgRoot=imgRootValWeb, classifierConfig=configCAMSwinSmallWeb, 
                                                classifierCheckpoint=checkpointCAMSwinSmallWeb, 
                                                annfile='../data/CompCars_web_original_split/meta/valKopie.txt',
                                                segData=segDataPathNoScaleWeb, segConfig=configSeg,
                                                segCheckpoint=checkpointSeg,  
                                                saveDir='swinSmall', 
                                                eval_data_original=eval_data_web_swinsmall, 
                                                imgRootBlurred=blurBackgroundImgRootWeb)

Evaluating original model vs blurred background.
Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at swinBase\annfile_filtered.txt
Using standard dataset on blurred data at directory ../data/CompCars_web_original_split/blurred/blurredBackground
Model already on GPU
Using given evaluation data of original model.
Loading data from Json G:/results/CompCars_Web/eval_results_swinbase.json
Computing Evaluation Metrics for Blurred Dataset
[>>>>>>>>>>>>>>>>>>>>>>>>] 15627/15627, 172.2 task/s, elapsed: 91s, ETA:     0s
Saving json data to swinBase\eval_results_blurred.json
Add total Change and improvement of original over blurred
Saving evaluation results to swinBase\evalBlurred.xlsx
Removing temporary filtered annotation file swinBase\annfile_filtered.txt
Evaluating original model vs blurred background.
Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at swinSmall\annfile_fil

In [ ]:
"""Evaluate Original Model (not retrained) on blurred Datasets"""
"""CompCars_Original Models"""
annfile=annfileValOriginal
cfgRes=configCAMResOriginal
cfgSwinBase=configCAMSwinBaseOriginal
cfgSwinSmall=configCAMSwinSmallOriginal
ckpRes=checkpointCAMResOriginal
ckpSwinBase=checkpointCAMSwinBaseOriginal
ckpSwinSmall=checkpointCAMSwinSmallOriginal
"""Background"""
fName = 'eval_results_blur_background'
imgDir = blurOGBackground
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Background, Hood, Front Bumper"""
fName = 'eval_results_blur_background_Hood_FBumper'
imgDir = blurOGBackground_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Front Glass, Hood, Front Bumper"""
fName = 'eval_results_blur_FGlass_Hood_FBumper'
imgDir = blurOGFGlass_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Front Left Light, Hood, Front Bumper"""
fName = 'eval_results_blur_FLLight_Hood_FBumper'
imgDir = blurOGFLLight_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Original/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)

Evaluating metrics on model.


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001D84628C820>
Traceback (most recent call last):
  File "C:\Users\Linus\anaconda3\envs\mmseg\lib\site-packages\torch\utils\data\dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "C:\Users\Linus\anaconda3\envs\mmseg\lib\site-packages\torch\utils\data\dataloader.py", line 1317, in _shutdown_workers
    self._mark_worker_as_unavailable(worker_id, shutdown=True)
  File "C:\Users\Linus\anaconda3\envs\mmseg\lib\site-packages\torch\utils\data\dataloader.py", line 1258, in _mark_worker_as_unavailable
    assert self._workers_status[worker_id] or (self._persistent_workers and shutdown)
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Created filtered annotation file at ./annfile_filtered.txt
[                                                  ] 0/13333, elapsed: 0s, ETA:

In [ ]:
"""Evaluate Original Model (not retrained) on blurred Datasets"""
"""CompCars_Color Models"""
annfile=annfileValColor
cfgRes=configCAMResColor
cfgSwinBase=configCAMSwinBaseColor
cfgSwinSmall=configCAMSwinSmallColor
ckpRes=checkpointCAMResColor
ckpSwinBase=checkpointCAMSwinBaseColor
ckpSwinSmall=checkpointCAMSwinSmallColor
"""Background"""
fName = 'eval_results_blur_background'
imgDir = blurColBackground
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Background, Hood, Front Bumper"""
fName = 'eval_results_blur_background_Hood_FBumper'
imgDir = blurColBackground_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Front Glass, Hood, Front Bumper"""
fName = 'eval_results_blur_FGlass_Hood_FBumper'
imgDir = blurColFGlass_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Front Left Light, Hood, Front Bumper"""
fName = 'eval_results_blur_FLLight_Hood_FBumper'
imgDir = blurColFLLight_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Color/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)

In [ ]:
"""Evaluate Original Model (not retrained) on blurred Datasets"""
"""CompCars_Web Models"""
annfile=annfileValWeb
cfgRes=configCAMResWeb
cfgSwinBase=configCAMSwinBaseWeb
cfgSwinSmall=configCAMSwinSmallWeb
ckpRes=checkpointCAMResWeb
ckpSwinBase=checkpointCAMSwinBaseWeb
ckpSwinSmall=checkpointCAMSwinSmallWeb
"""Background"""
fName = 'eval_results_blur_background'
imgDir = blurWebBackground
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/ResNet/blurred', 
                 fileName=fName, saveExcel=True)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/SwinBase/blurred', 
                 fileName=fName, saveExcel=True)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/SwinSmall/blurred', 
                 fileName=fName, saveExcel=True)
"""Back Bumper, Background, Front Bumper"""
fName = 'eval_results_blur__BBumper_background_FBumper'
imgDir = blurWebBBumper_Background_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/ResNet/blurred', 
                 fileName=fName)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/SwinBase/blurred', 
                 fileName=fName)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/SwinSmall/blurred', 
                 fileName=fName)
"""Background, Hood, Front Bumper"""
fName = 'eval_results_blur_Background_Hood_FBumper'
imgDir = blurWebBackground_Hood_FBumper
# ResNet
get_eval_metrics(cfgRes, ckpRes, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/ResNet/blurred', 
                 fileName=fName)
# SwinBase
get_eval_metrics(cfgSwinBase, ckpSwinBase, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/SwinBase/blurred', 
                 fileName=fName)
# SwinSmall
get_eval_metrics(cfgSwinSmall, ckpSwinSmall, imgDir, 
                 annfile, saveDir='evaluations/CompCars_Web/SwinSmall/blurred', 
                 fileName=fName)